In [1]:
import gmaps
import pandas as pd
import requests

from config import g_key

In [2]:
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Chokurdakh,RU,2022-10-05 19:22:55,70.6333,147.9167,16.29,97,97,4.83
1,1,Hithadhoo,MV,2022-10-05 19:22:56,-0.6000,73.0833,82.40,73,100,17.56
2,2,Hilo,US,2022-10-05 19:22:57,19.7297,-155.0900,85.28,74,40,3.44
3,3,La Ronge,CA,2022-10-05 19:22:57,55.1001,-105.2842,43.02,45,40,6.91
4,4,Busselton,AU,2022-10-05 19:22:58,-33.6500,115.3333,49.48,60,18,9.91


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use API key
gmaps.configure(api_key=g_key)

In [7]:
# Heatmap of temperature
# Get lat and lng
locations = city_data_df[['Lat', 'Lng']]
# Get max temp
max_temp = city_data_df['Max Temp']
# Assign figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=[max(temp, 0) for temp in max_temp],
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[['Lat', 'Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=clouds,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of wind speed
locations = city_data_df[['Lat', 'Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=wind,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the user to add a min and max temp value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# Filter the dataset to find cities that fit criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)].dropna()

preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Hithadhoo,MV,2022-10-05 19:22:56,-0.6000,73.0833,82.40,73,100,17.56
2,2,Hilo,US,2022-10-05 19:22:57,19.7297,-155.0900,85.28,74,40,3.44
5,5,Kapaa,US,2022-10-05 19:21:29,22.0752,-159.3190,80.58,87,100,5.75
21,21,Wukari,NG,2022-10-05 19:23:10,7.8500,9.7833,77.61,84,55,3.36
25,25,Cartagena,CO,2022-10-05 19:20:23,10.3997,-75.5144,82.02,78,75,11.50
37,37,Butaritari,KI,2022-10-05 19:23:22,3.0707,172.7902,81.64,72,6,20.20
38,38,Vaini,TO,2022-10-05 19:23:23,-21.2000,-175.2000,76.51,86,100,13.96
39,39,Mocuba,MZ,2022-10-05 19:23:24,-16.8392,36.9856,75.99,63,0,7.07
43,43,Faanui,PF,2022-10-05 19:23:27,-16.4833,-151.7500,77.81,75,51,26.24
44,44,Wewak,PG,2022-10-05 19:23:27,-3.5534,143.6268,79.68,85,100,7.31


In [21]:
preferred_cities_df.isna().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [22]:
# Create df called hotel_df to store hotel names with city, country, max temp, and coords
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hithadhoo,MV,82.40,-0.6000,73.0833,
2,Hilo,US,85.28,19.7297,-155.0900,
5,Kapaa,US,80.58,22.0752,-159.3190,
21,Wukari,NG,77.61,7.8500,9.7833,
25,Cartagena,CO,82.02,10.3997,-75.5144,
37,Butaritari,KI,81.64,3.0707,172.7902,
38,Vaini,TO,76.51,-21.2000,-175.2000,
39,Mocuba,MZ,75.99,-16.8392,36.9856,
43,Faanui,PF,77.81,-16.4833,-151.7500,
44,Wewak,PG,79.68,-3.5534,143.6268,


In [23]:
# Set parameters for hotel search
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [25]:
# Iterate through df
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # Add lat and lng to location key in param dictionary
    params['location'] = f'{lat},{lng}'
    
    # Use the search term lodging and location
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # Make request and get JSON data
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hithadhoo,MV,82.40,-0.6000,73.0833,Scoop Guest House
2,Hilo,US,85.28,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Kapaa,US,80.58,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
21,Wukari,NG,77.61,7.8500,9.7833,Elimtop
25,Cartagena,CO,82.02,10.3997,-75.5144,Hotel Colombia Real - Cartagena
37,Butaritari,KI,81.64,3.0707,172.7902,Isles Sunset Lodge
38,Vaini,TO,76.51,-21.2000,-175.2000,Keleti Beach Resort
39,Mocuba,MZ,75.99,-16.8392,36.9856,Phama Lodge
43,Faanui,PF,77.81,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
44,Wewak,PG,79.68,-3.5534,143.6268,In Wewak Boutique Hotel


In [29]:
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</d1>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temps for the vacation spots
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,
                                  info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))